# Решение СЛАУ

In [1]:
import numpy as np
from numpy import *
import matplotlib.pyplot as plt
from math import *
import sympy

## Входные данные


<div>
<img src="system.png" width="600"/>
</div>

In [258]:
# It's not fast... But universal :)

def cell(i, j):
    if i == j:
        return 10
    elif j > (i + 1):
        return 0
    else:
        return 1 / (i + 1)
    
def fv(i):
    return i + 1

n = 4  

In [259]:
matrix = np.fromfunction(np.vectorize(cell, otypes=[float]), (n, n), dtype=float)
matrix

array([[10.        ,  1.        ,  0.        ,  0.        ],
       [ 0.5       , 10.        ,  0.5       ,  0.        ],
       [ 0.33333333,  0.33333333, 10.        ,  0.33333333],
       [ 0.25      ,  0.25      ,  0.25      , 10.        ]])

In [260]:
f = np.fromfunction(np.vectorize(fv, otypes=[float]), (n,), dtype=float)
f

array([1., 2., 3., 4.])

### Определим норму и $\varepsilon$ для прямых методов

In [244]:
epsilon = 1e-6

def norm(x):
    return np.max(abs(x))

### Метод Гаусса с выбором главного элемента

In [251]:
def gauss(matrix, f):

    # Соберем всё в одну матрицу M = [A|f]
    M = np.hstack((matrix, f.reshape(-1, 1)))   

    # Прямой ход
    for i in range(len(matrix)):

        # Главный элемент отправляется на соответствующую строку
        pivot = np.argmax(np.abs(matrix[:,i]))
        M[[i, pivot]] = M[[pivot, i]] 

        div = M[i][i]

        # "Нормируем" строку
        row = M[i]
        row /= div

        # Зануляем всю колонку под нами
        for down in M[i + 1:]:
            down -= down[i] * row

    # Обратный ход
    for i in range(len(matrix) - 1, 0, -1):
        row = M[i]
        for up in reversed(M[:i]):
            up -= up[i] * row

    # Получена матрица вида M = [E|x]. Вытаскиваем x
    return M[:,-1]

In [252]:
x = gauss(matrix, f)

diff = np.matmul(matrix, x.T) - f
assert norm(diff) < epsilon

### LU-разложение

In [336]:
def lu(A):
    U = A.copy()
    L = np.eye(n, dtype=np.double)
    
    for i in range(len(A)):
        
        # Чистим столбец от диагонали вниз.
        # Затем делаем обратное преобразование. Awesome!
        factor = U[i+1:, i] / U[i, i]
        L[i+1:, i] = factor
        U[i+1:] -= factor.reshape(-1, 1) * U[i]
        
    return L, U


L,U = lu(matrix)
assert norm(np.matmul(L, U) - matrix) < epsilon
np.matmul(L, U) - matrix

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

### Метод Зейделя

### Метод Якоби

### Метод верхней релаксации